In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
data=yf.download(tickers='HAL.NS',period='max',interval='1d')
historical_stock_data=data[['Adj Close']]

# type(historical_stock_data) is a data frame as data is dataframe

[*********************100%%**********************]  1 of 1 completed


In [2]:
historical_stock_data['pct_chnage']=historical_stock_data['Adj Close'].pct_change().dropna()
mean=historical_stock_data['pct_chnage'].mean()
std=historical_stock_data['pct_chnage'].std()


q25 = np.percentile(historical_stock_data['pct_chnage'].dropna(), 25)
q50 = np.percentile(historical_stock_data['pct_chnage'].dropna(), 50)
q75 = np.percentile(historical_stock_data['pct_chnage'].dropna(), 75)
print("Q1:", q25)
print("Q2:", q50)
print("Q3:", q75)
historical_stock_data.head()


Q1: -0.010546141367069273
Q2: -0.000408005557446911
Q3: 0.011527538003877513


<ipython-input-2-814c065c4418>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_stock_data['pct_chnage']=historical_stock_data['Adj Close'].pct_change().dropna()


,Adj Close,pct_chnage
Date,,
2018-04-02,480.766510,NaN
2018-04-03,472.128052,-0.017968
2018-04-04,466.815186,-0.011253
2018-04-05,465.999481,-0.001747
2018-04-06,461.356079,-0.009964


In [3]:
# Initialize counters
ll = ml = hl = lm = mm = hm = hh = lh = mh = 0

# Iterate through the data to count transitions
for i in range(1, len(historical_stock_data)):
    prev_change = historical_stock_data.iloc[i-1, 1]
    curr_change = historical_stock_data.iloc[i, 1]

    if prev_change < q25 and curr_change < q25:
        ll += 1
    if q25 <= prev_change <= q75 and curr_change < q25:
        ml += 1
    if prev_change > q75 and curr_change < q25:
        hl += 1
    if prev_change < q25 and q25 <= curr_change <= q75:
        lm += 1
    if q25 <= prev_change <= q75 and q25 <= curr_change <= q75:
        mm += 1
    if prev_change > q75 and q25 <= curr_change <= q75:
        hm += 1
    if prev_change > q75 and curr_change > q75:
        hh += 1
    if prev_change < q25 and curr_change > q75:
        lh += 1
    if q25 <= prev_change <= q75 and curr_change > q75:
        mh += 1

# Compute the transition matrix
transition_matrix_for_markov_chains = np.array([
    [ll / (ll + lm + lh), lm / (ll + lm + lh), lh / (ll + lm + lh)],
    [ml / (ml + mm + mh), mm / (ml + mm + mh), mh / (ml + mm + mh)],
    [hl / (hl + hm + hh), hm / (hl + hm + hh), hh / (hl + hm + hh)]
])



In [4]:
transition_matrix_for_markov_chains

array([[0.31887755, 0.44132653, 0.23979592],
       [0.22094508, 0.56066411, 0.2183908 ],
       [0.23785166, 0.43734015, 0.32480818]])

In [5]:
import numpy as np

# Historical data properties
mean_return = mean  # Mean daily return (0.05%)
std_dev = std # Standard deviation of daily returns (1%)

# Define transition matrix (example)
P = transition_matrix_for_markov_chains

# Define the states and initial state
states = ['D1', 'S', 'I1']
initial_state = 'S'
initial_price = 4670

# Map states to index
state_index = {'D1': 0, 'S': 1, 'I1': 2}

# Define function to get next state
def get_next_state(current_state):
    return np.random.choice(states, p=P[state_index[current_state]])

# Define function to get price change based on state
def get_price_change(state):
    if state == 'D1':
        return np.random.normal(q25, std_dev)  # Decrease
    elif state == 'S':
        return np.random.normal(q50, std_dev)  # Neutral
    elif state == 'I1':
        return np.random.normal(q75, std_dev)  # Increase

# Define function to simulate price path
def simulate_price_path(n_days):
    current_state = initial_state
    current_price = initial_price
    price_path = [current_price]

    for _ in range(n_days):
        next_state = get_next_state(current_state)
        price_change = get_price_change(next_state)
        current_price *= (1 + price_change)
        price_path.append(current_price)
        current_state = next_state

    return price_path[-1]

# Run simulation 1000 times
n_simulations = 1000
n_days=17
all_price_paths = np.array([simulate_price_path(n_days) for _ in range(n_simulations)])

# Calculate statistics
mean_prices = np.mean(all_price_paths)


# Print the results
print("Mean Prices:")
print(mean_prices)

print("5th percentile:")
print(np.percentile(all_price_paths, 5))
print("95th percentile:")
print(np.percentile(all_price_paths, 95))


Mean Prices:
4672.133003055975
Min Prices:
3392.0428099377714
Max Prices:
6364.218274664304
5th percentile:
3921.230205176004
95th percentile:
5461.440816527346
